In [1]:
import pandas as pd
import numpy as np
from fastDamerauLevenshtein import damerauLevenshtein
import ast
import math

In [2]:
class Prediction():
    Item = None
    Parent = None
    Children = None

    def __init__(self, itemValue=None):
        self.Item = itemValue
        self.Children = []
        self.Parent = None

    def get_child(self, target):
        for chld in self.Children:
            if chld.Item == target:
                return chld
        return None

    def get_children(self):
        return self.Children

    def has_child(self, target):
        found = self.get_child(target)
        if found is not None:
            return True
        else:
            return False

    def add_new_child(self, child):
        newchild = Prediction(child)
        newchild.Parent = self
        self.Children.append(newchild)

    def remove_child(self, child):
        for chld in self.Children:
            if chld.Item == child:
                self.Children.remove(chld)

In [3]:
class CPT():
    alphabet = None
    root = None
    II = None
    LT = None
    def __init__(self):
        self.alphabet = set()
        self.root = Prediction()
        self.II = {}
        self.LT = {}

    def load_files(self, train_file, test_file=None):
        """
        This function reads in the wide csv file of sequences separated by commas and returns a list of list of 
        those sequences. The sequences are defined as below.
        seq1 = A,B,C,D
        seq2  B,C,E
        Returns: [[A,B,C,D],[B,C,E]]
        """
        
        data = []
        target = []

        if train_file is None:
            return train_file

        train = pd.read_csv(train_file, header=0)

        for index, row in train.iterrows():
            data.append(row.values)

        if test_file is not None:

            test = pd.read_csv(test_file, header=None)

            for index, row in test.iterrows():
                data.append(row.values)
                target.append(list(row.values))

            return data, target

        return data


    def train(self, data):
        """
         This functions populates the Prediction Tree, Inverted Index and LookUp Table for the algorithm.
         Input: The list of list training data
         Output : Boolean True
         """

        cursornode = self.root

        for seqid, row in enumerate(data):
            for element in row:
                if element == element: # different sequence length support
                    
                    if cursornode.has_child(element) == False:
                        cursornode.add_new_child(element)
                        cursornode = cursornode.get_child(element)
                    else:
                        cursornode = cursornode.get_child(element)

                # adding to the inverted index

                    if self.II.get(element) is None:
                        self.II[element] = set()

                    self.II[element].add(seqid)

                    self.alphabet.add(element)

            self.LT[seqid] = cursornode

            cursornode = self.root

        return True

    def score(self, counttable, key, length, target_size, number_of_similar_sequences, number_items_counttable):
        """
         This functions populates the Prediction Tree, Inverted Index and LookUp Table for the algorithm.
         Input: The list of list training data
         Output : Boolean True
         """
        
        weight_level = 1 / number_of_similar_sequences
        weight_distance = 1 / number_items_counttable
        score = 1 + weight_level + weight_distance * 0.001

        if counttable.get(key) is None:
            counttable[key] = score
        else:
            counttable[key] = score * counttable.get(key)

        return counttable

    def predict(self, data, target, k, n):
        """
        Here target is the test dataset in the form of list of list,
        k is the number of last elements that will be used to find similar sequences and,
        n is the number of predictions required.
        Input: training list of list, target list of list, k,n
        Output: max n predictions for each sequence
        """

        predictions = []

        for index, each_target in enumerate(target):
            #n = int(n_list[index])
            
            # different sequence length support
            i = 0
            while i < len(each_target) and each_target[i] == each_target[i]:  # find NaN start
                i = i + 1
            l = i - k - 1
            if l < 0:
                l = 0
            
            each_target = each_target[l:i]

            intersection = set(range(0, len(data)))

            for element in each_target:
                if self.II.get(element) is None:
                    continue
                intersection = intersection & self.II.get(element)

            similar_sequences = []

            for element in intersection:
                currentnode = self.LT.get(element)
                tmp = []
                while currentnode.Item is not None:
                    tmp.append(currentnode.Item)
                    currentnode = currentnode.Parent
                similar_sequences.append(tmp)

            for sequence in similar_sequences:
                sequence.reverse()

            counttable = {}

            for sequence in similar_sequences:
                try:
                    index = next(
                        i for i, v in zip(range(len(sequence) - 1, 0, -1), reversed(sequence)) if v == each_target[-1])
                except:
                    index = None
                if index is not None:
                    count = 1
                    for element in sequence[index + 1:]:
                        if element in each_target:
                            continue

                        counttable = self.score(counttable, element, len(each_target), len(each_target),
                                                len(similar_sequences), count)
                        count += 1

            pred = self.get_n_largest(counttable, n)
            predictions.append(pred)

        return predictions

    def get_n_largest(self, dictionary, n):

        largest = sorted(dictionary.items(), key=lambda t: t[1], reverse=True)[:n]
        return [key for key, _ in largest]

In [4]:
model = CPT()

In [5]:
data, target = model.load_files('data/all_data_train.csv', 'data/all_data_test_7.csv')

In [6]:
model.train(data)

True

In [7]:
# get length of each sequence (ignoring nan) minus given sequence in test
n_list = [np.count_nonzero(~pd.isnull(x)) - 1 for x in data]
n_list = [x if x > 0 else 0 for x in n_list]

In [12]:
predictions = model.predict(data, target, 7, 1)

In [13]:
i = 0
distances = []

for elem in predictions:
    pred = [x for x in target[i] if str(x) != 'nan']
    dat = [x for x in data[i] if str(x) != 'nan']
    predicted = str(''.join(pred) + ''.join(elem))
    observed = ''.join(dat)
    dl = 1 - damerauLevenshtein(predicted, observed)
    distances.append(dl)
    print(target[i])
    print('for sequence {0} prediction is {1}'.format(i, elem))
    print('edit distance between {0} and {1}: {2}'.format(observed, predicted, dl))
    i += 1
    print('----------')

print('Average similarity: {0}'.format(np.mean(distances)))
print('Median similarity: {0}'.format(np.median(distances)))

['p', 'o', 'c', 'g', 'k', 'r', nan]
for sequence 0 prediction is ['f']
edit distance between pocgkr and pocgkrf: 0.1428571428571429
----------
['c', 'g', 'w', 'p', 'c', 'f', 'k']
for sequence 1 prediction is ['s']
edit distance between cgwpcfks and cgwpcfks: 0.0
----------
['k', 'f', 's', 'f', 'k', 's', 'p']
for sequence 2 prediction is ['g']
edit distance between kfsfkspwg and kfsfkspg: 0.11111111111111116
----------
['p', 'f', 'k', 's', 'w', 'k', 'f']
for sequence 3 prediction is ['r']
edit distance between pfkswkfsococg and pfkswkfr: 0.46153846153846156
----------
['w', 'p', 't', 'g', 'k', 'f', 's']
for sequence 4 prediction is ['c']
edit distance between wptgkfsoc and wptgkfsc: 0.11111111111111116
----------
['p', 'w', 'f', 'k', 'g', nan, nan]
for sequence 5 prediction is ['s']
edit distance between pwfkg and pwfkgs: 0.16666666666666663
----------
['p', 'o', 'w', 'c', 'e', 'r', nan]
for sequence 6 prediction is ['k']
edit distance between powcer and powcerk: 0.1428571428571429
----

### Prequential method

In [10]:
ones = [1 for x in data]

In [11]:
predictions = model.predict(data, target, 7, 1)


In [73]:
def get_string(string):
    output = ''
    for lst in string:
        for elem in lst:
            output += elem
    return output

In [161]:
errors = [[] for x in range(0,len(data)//2)]

for seq in range(0,len(data)//2):
    i = 1
    while i <= n_list[seq]:
        next_char = model.predict(data, target[seq][:1], 3, 1)
        #print('next char: ' , next_char)
        next_char = get_string(next_char)
        predicted = str(''.join(data[seq][:i]) + next_char)
        #print('pred: ', predicted)
        observed = ''.join(data[seq][:i+1])
        #print('observed: ', observed)
        error = 1 - damerauLevenshtein(predicted, observed)
        #print(error, '\n' , '____________')
        
        errors[seq].append(error)
        
        i += 1

In [162]:
error_list = [np.median(error) for error in errors]

In [163]:
np.mean([np.median(error) for error in errors])

0.19557890412057072

In [164]:
np.median([np.median(error) for error in errors])

0.16666666666666663

In [170]:
with open('results/cpt_prequential.txt', 'w') as f:
    f.write(str(error_list))

### Binary evaluation of errors in each step

In [173]:
errors = [[] for x in range(0,len(data)//2)]

for seq in range(0,len(data)//2):
    i = 1
    while i <= n_list[seq]:
        next_char = model.predict(data, target[seq][:1], 3, 1)
        #print('next char: ' , next_char)
        next_char = get_string(next_char)
        predicted = next_char
        #print('pred: ', predicted)
        #observed = ''.join(data[seq][:i+1])
        observed = data[seq][i]
        #print('observed: ', observed)
        error = 1 - damerauLevenshtein(predicted, observed)
        #print(error, '\n' , '____________')
        
        errors[seq].append(error)
        
        i += 1

In [176]:
summed_error = [sum(error) for error in errors]
np.median(summed_error)

6.0

In [175]:
with open('results/cpt_prequential_summed.txt', 'w') as f:
    f.write(str(summed_error))